In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_file = "/content/drive/MyDrive/FYP/validCheck.zip"
#train_file = "/content/drive/MyDrive/FYP/p.h.r.dataset.All.zip"

import zipfile

with zipfile.ZipFile(train_file, 'r') as z:
    z.extractall()

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras import models
from keras.preprocessing.image import ImageDataGenerator
import os
import cv2

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (224, 224, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())

model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 93312)             0         
                                                                 
 dense (Dense)               (None, 128)               1

In [ ]:
# ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# 
training_set = train_datagen.flow_from_directory('validCheck/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'binary')


Found 1806 images belonging to 2 classes.


In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
        'validCheck/validation',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary',
        shuffle = False
)

Found 302 images belonging to 2 classes.


In [ ]:
validation_generator.class_indices

{'invalid': 0, 'valid': 1}

In [ ]:
model.fit(training_set,
                         steps_per_epoch = 40,
                         epochs = 7,
                         validation_steps = 20,
                         validation_data=validation_generator
                    )

print(" trained Successfully!")

Epoch 1/5
40/40 [==============================] - 100s 2s/step - loss: 0.0517 - accuracy: 0.9828 - val_loss: 0.0969 - val_accuracy: 0.9768
Epoch 2/5
40/40 [==============================] - 98s 2s/step - loss: 0.0311 - accuracy: 0.9898 - val_loss: 0.0834 - val_accuracy: 0.9768
Epoch 3/5
40/40 [==============================] - 97s 2s/step - loss: 0.0470 - accuracy: 0.9842 - val_loss: 0.1006 - val_accuracy: 0.9834
Epoch 4/5
40/40 [==============================] - 97s 2s/step - loss: 0.0332 - accuracy: 0.9881 - val_loss: 0.0963 - val_accuracy: 0.9801
Epoch 5/5
40/40 [==============================] - 98s 2s/step - loss: 0.0398 - accuracy: 0.9852 - val_loss: 0.1509 - val_accuracy: 0.9768
 trained Successfully!


In [ ]:
model.evaluate(validation_generator)

10/10 [==============================] - 8s 790ms/step - loss: 0.1509 - accuracy: 0.9768


[0.1508782058954239, 0.9768211841583252]

In [ ]:
model.save('model1.h5')

In [ ]:
model=models.load_model('/content/model1.h5')


image = cv2.imread('/content/validCheck/validation/invalid/articfox (100).jpg')
image = cv2.resize(image, (224,224))
image = image.reshape(1, 224, 224, 3)

result = model.predict(image)
if(result[0][0] == 1):
    print("valid!")
    print(result)
else:
    print("invalid!")
    print(result)

invalid!
[[0.]]


testing 2

In [ ]:
from keras.models import model_from_json
# Convert the Model to json
model_json = model.to_json()
with open("model.json","w") as json_file:
  json_file.write(model_json)

# Save the weights in a seperate file
model.save_weights("modelCheck.h5")

In [ ]:

os.stat('/content/model1.h5').st_size

143497584

In [ ]:
!cp /content/model1.h5 /content/drive/MyDrive/FYP/

In [ ]:
training_set.class_indices

{'invalid': 0, 'valid': 1}

In [ ]:


json_file = open('/content/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model1 = model_from_json(loaded_model_json)

# Load the weights
loaded_model1.load_weights("/content/modelCheck.h5")

image = cv2.imread('/content/articfox (93).jpg')
image = cv2.resize(image, (224,224))
image = image.reshape(1, 224, 224, 3)

result = loaded_model1.predict(image)
if(result[0][0] == 1):
    print("valid!")
else:
    print("invalid!")

invalid!
